In [189]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
from sportsipy.nfl.boxscore import Boxscores, Boxscore
from sportsipy.nfl.schedule import Schedule, Game
from sportsipy.nfl.teams import Teams
pd.set_option('display.max_columns', None)

In [190]:
# Import dataframes
test_df = pd.read_csv(Path('Resources/test_df.csv'), index_col='Unnamed: 0')
train_df = pd.read_csv(Path('Resources/train_df.csv'), index_col='Unnamed: 0')
overall_df = pd.read_csv(Path('Resources/overall_df.csv'), index_col='Unnamed: 0')

In [191]:
pred_week = 12
current_week = pred_week
train_df = overall_df[overall_df['week'] < pred_week]
test_df = overall_df[overall_df['week'] == pred_week]

In [192]:
train_df_info = train_df
test_df_info = test_df

display(train_df.tail())
display(test_df.tail())

,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
143,Cincinnati Bengals,cin,Las Vegas Raiders,rai,11,0.000000,-1.777778,-0.333333,0.333333,-34.555556,-6.333333,-4.111111,0.666667,-30.000000,-3.222222,2.888889,2.000000,0.000000,12.111111,24.333333,0.555556,-22.444444,0.333333,-28.888889,4.555556,0.076923,0.010260,1.0,6.065577,-41.499542
144,Arizona Cardinals,crd,Seattle Seahawks,sea,11,0.466667,4.266667,1.022222,0.355556,47.255556,2.311111,3.155556,0.133333,41.322222,0.833333,8.588889,6.844444,0.822222,26.022222,319.611111,-0.822222,73.277778,0.433333,12.422222,-5.933333,0.466667,0.091837,1.0,75.434931,-53.283848
145,Dallas Cowboys,dal,Kansas City Chiefs,kan,11,0.177778,-1.122222,-0.144444,-0.333333,-0.766667,-6.011111,-2.633333,-0.055556,1.088889,1.288889,5.355556,5.711111,0.400000,29.055556,49.766667,-0.144444,28.288889,-0.666667,13.922222,1.855556,-0.165775,-0.076437,0.0,-93.812538,16.993555
146,Pittsburgh Steelers,pit,Los Angeles Chargers,sdg,11,0.069444,-3.111111,0.666667,-0.222222,-33.555556,-0.444444,-0.555556,-0.888889,-35.222222,-0.111111,-4.666667,2.333333,-0.222222,-5.555556,196.555556,0.222222,-39.111111,0.222222,-13.444444,-1.666667,-0.283422,-0.037022,0.0,12.663172,-34.675099
147,New York Giants,nyg,Tampa Bay Buccaneers,tam,11,-0.333333,-3.777778,0.444444,0.000000,-76.666667,-7.000000,-5.666667,-2.000000,-70.666667,-0.777778,-11.111111,3.111111,0.000000,5.000000,-66.111111,0.888889,-71.666667,-0.111111,-17.555556,6.000000,-0.038462,-0.089294,0.0,-192.654842,-128.711909


,away_name,away_abbr,home_name,home_abbr,week,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
158,Los Angeles Chargers,sdg,Denver Broncos,den,12,0.100000,3.800000,-0.100000,0.200000,49.800000,5.300000,2.700000,0.8,42.500000,1.800000,6.000000,-1.600000,0.300000,-5.400000,-163.900000,-1.200000,44.400000,-0.100000,16.600000,-7.300000,-0.055556,0.105924,0.0,74.721833,99.202771
159,Minnesota Vikings,min,San Francisco 49ers,sfo,12,0.000000,0.100000,0.000000,-0.400000,31.700000,7.400000,6.100000,0.6,32.000000,1.400000,0.900000,-1.100000,-0.700000,-4.000000,41.600000,-0.500000,27.700000,-0.800000,5.300000,0.300000,0.155844,0.009179,0.0,6.369632,92.115953
160,Los Angeles Rams,ram,Green Bay Packers,gnb,12,-0.027273,0.700000,-0.490909,0.445455,46.300000,3.154545,2.609091,0.4,44.918182,0.163636,4.645455,-1.336364,0.063636,-10.100000,-205.163636,-0.500000,36.200000,0.281818,-3.627273,-1.381818,-0.071795,-0.002783,0.0,-77.381374,-42.459544
161,Cleveland Browns,cle,Baltimore Ravens,rav,12,-0.154545,-3.772727,-0.027273,-0.354545,-45.963636,-6.236364,-4.272727,-0.4,-47.590909,1.472727,-2.518182,-1.690909,0.345455,5.818182,-187.427273,-0.581818,-40.145455,-0.300000,15.800000,-1.627273,-0.257576,0.020200,0.0,-51.409517,-103.082625
162,Seattle Seahawks,sea,Washington Football Team,was,12,-0.100000,-4.100000,-0.700000,-0.500000,-24.700000,-4.800000,-3.100000,0.0,-13.700000,-0.500000,-1.800000,-5.000000,0.100000,-24.500000,-336.400000,1.000000,-49.200000,-0.900000,-9.300000,11.000000,-0.355072,-0.068234,0.0,26.031709,32.264597


In [193]:
# Drop non-numerical columns
train_df = train_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
test_df = test_df.drop(columns=['away_name','away_abbr','home_name','home_abbr','week'])
display(test_df.head())
display(train_df.tail())

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
148,0.300000,-1.600000,0.500000,0.100000,-42.000000,-8.700000,-7.300000,-0.100000,-37.100000,-0.400000,0.300000,4.100000,0.000000,17.900000,-20.600000,0.900000,-24.100000,0.100000,0.500000,4.900000,-7.763975e-02,0.010196,1.0,146.190723,1.927580
149,-0.200000,-4.300000,-0.300000,0.100000,5.300000,-0.100000,-0.500000,-0.600000,8.100000,-0.200000,-7.000000,-6.200000,-0.100000,-50.100000,-113.400000,0.300000,-44.800000,-0.300000,2.500000,2.800000,-9.049774e-03,-0.086816,1.0,-95.585615,-39.040283
150,0.100000,3.300000,0.900000,0.300000,69.900000,7.700000,7.200000,-0.100000,69.300000,1.400000,4.400000,-3.400000,0.300000,0.900000,43.700000,-0.200000,70.800000,0.300000,15.100000,-0.600000,-2.833333e-01,0.104856,1.0,22.731979,131.385591
151,0.154545,1.336364,-0.127273,-0.445455,-30.445455,-6.636364,-5.009091,0.181818,-35.063636,0.709091,8.100000,7.427273,0.845455,60.463636,8.845455,-0.381818,30.018182,-0.581818,4.772727,-4.618182,-1.041667e-01,0.087977,0.0,81.771113,66.528163
152,0.090909,0.818182,-0.090909,0.363636,-10.454545,1.090909,-0.636364,-0.181818,-7.454545,0.818182,-0.818182,2.727273,0.272727,12.545455,116.181818,0.818182,2.272727,0.181818,7.909091,3.000000,-1.110223e-16,-0.052341,0.0,1.748849,34.094641


,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
143,0.000000,-1.777778,-0.333333,0.333333,-34.555556,-6.333333,-4.111111,0.666667,-30.000000,-3.222222,2.888889,2.000000,0.000000,12.111111,24.333333,0.555556,-22.444444,0.333333,-28.888889,4.555556,0.076923,0.010260,1.0,6.065577,-41.499542
144,0.466667,4.266667,1.022222,0.355556,47.255556,2.311111,3.155556,0.133333,41.322222,0.833333,8.588889,6.844444,0.822222,26.022222,319.611111,-0.822222,73.277778,0.433333,12.422222,-5.933333,0.466667,0.091837,1.0,75.434931,-53.283848
145,0.177778,-1.122222,-0.144444,-0.333333,-0.766667,-6.011111,-2.633333,-0.055556,1.088889,1.288889,5.355556,5.711111,0.400000,29.055556,49.766667,-0.144444,28.288889,-0.666667,13.922222,1.855556,-0.165775,-0.076437,0.0,-93.812538,16.993555
146,0.069444,-3.111111,0.666667,-0.222222,-33.555556,-0.444444,-0.555556,-0.888889,-35.222222,-0.111111,-4.666667,2.333333,-0.222222,-5.555556,196.555556,0.222222,-39.111111,0.222222,-13.444444,-1.666667,-0.283422,-0.037022,0.0,12.663172,-34.675099
147,-0.333333,-3.777778,0.444444,0.000000,-76.666667,-7.000000,-5.666667,-2.000000,-70.666667,-0.777778,-11.111111,3.111111,0.000000,5.000000,-66.111111,0.888889,-71.666667,-0.111111,-17.555556,6.000000,-0.038462,-0.089294,0.0,-192.654842,-128.711909


In [194]:
# Separate Features from labels.
X_train = train_df.drop(columns='result')
y_train = train_df['result']
X_test = test_df.drop(columns='result')
y_test = test_df['result']

display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())
X_train.shape

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
0,0.0,4.0,-2.0,0.0,121.0,16.0,8.0,0.0,132.0,-4.0,-3.0,-7.0,1.0,-66.0,55.0,1.0,55.0,0.0,-9.0,11.0,-0.666667,0.200000,-50.848658,100.316973
1,1.0,-4.0,-1.0,-1.0,29.0,-13.0,-9.0,2.0,45.0,0.0,13.0,10.0,-1.0,15.0,-23.0,2.0,44.0,-2.0,-20.0,16.0,0.666667,-0.240260,-51.742731,34.616025
2,0.0,4.0,-1.0,0.0,-119.0,-14.0,-9.0,4.0,-128.0,0.0,19.0,12.0,-1.0,60.0,196.0,-1.0,-59.0,-1.0,-2.0,-9.0,1.000000,0.214286,204.627654,54.983874
3,0.0,10.0,0.0,1.0,232.0,24.0,16.0,1.0,247.0,5.0,10.0,0.0,2.0,7.0,445.0,1.0,239.0,1.0,30.0,15.0,0.000000,0.133333,-62.282436,-14.386838
4,0.0,-3.0,0.0,0.0,50.0,-9.0,-9.0,-1.0,55.0,-7.0,9.0,-3.0,1.0,-42.0,-270.0,0.0,8.0,2.0,-10.0,5.0,0.000000,-0.128205,36.874709,-13.318243


0    0.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: result, dtype: float64

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,elo_dif,qb_dif
148,0.300000,-1.600000,0.500000,0.100000,-42.000000,-8.700000,-7.300000,-0.100000,-37.100000,-0.400000,0.300000,4.100000,0.000000,17.900000,-20.600000,0.900000,-24.100000,0.100000,0.500000,4.900000,-7.763975e-02,0.010196,146.190723,1.927580
149,-0.200000,-4.300000,-0.300000,0.100000,5.300000,-0.100000,-0.500000,-0.600000,8.100000,-0.200000,-7.000000,-6.200000,-0.100000,-50.100000,-113.400000,0.300000,-44.800000,-0.300000,2.500000,2.800000,-9.049774e-03,-0.086816,-95.585615,-39.040283
150,0.100000,3.300000,0.900000,0.300000,69.900000,7.700000,7.200000,-0.100000,69.300000,1.400000,4.400000,-3.400000,0.300000,0.900000,43.700000,-0.200000,70.800000,0.300000,15.100000,-0.600000,-2.833333e-01,0.104856,22.731979,131.385591
151,0.154545,1.336364,-0.127273,-0.445455,-30.445455,-6.636364,-5.009091,0.181818,-35.063636,0.709091,8.100000,7.427273,0.845455,60.463636,8.845455,-0.381818,30.018182,-0.581818,4.772727,-4.618182,-1.041667e-01,0.087977,81.771113,66.528163
152,0.090909,0.818182,-0.090909,0.363636,-10.454545,1.090909,-0.636364,-0.181818,-7.454545,0.818182,-0.818182,2.727273,0.272727,12.545455,116.181818,0.818182,2.272727,0.181818,7.909091,3.000000,-1.110223e-16,-0.052341,1.748849,34.094641


148    1.0
149    1.0
150    1.0
151    0.0
152    0.0
Name: result, dtype: float64

(148, 24)

In [195]:
# Scale the X data
# Instantiate the scaler
X_scaler = StandardScaler()

# Fit the scaler to Train data
X_scaler.fit(X_train)

#Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [196]:
# Build the NN model.
number_input_features = len(X_train.columns)

hidden_nodes_layer1 = (number_input_features + 1)//2

hidden_nodes_layer2 = (hidden_nodes_layer1 + 1)//2

hidden_nodes_layer3 = (hidden_nodes_layer2 + 1)//2

display((f'Number of input features: {number_input_features}'))
display((f'Layer 1: {hidden_nodes_layer1}'))
display((f'Layer 2: {hidden_nodes_layer2}'))
display((f'Layer 3: {hidden_nodes_layer3}'))

'Number of input features: 24'

'Layer 1: 12'

'Layer 2: 6'

'Layer 3: 3'

In [197]:
# Create Sequential NN_model
nn = Sequential()

#Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third layer
nn.add(Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [198]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 12)                300       
                                                                 
 dense_41 (Dense)            (None, 6)                 78        
                                                                 
 dense_42 (Dense)            (None, 3)                 21        
                                                                 
 dense_43 (Dense)            (None, 1)                 4         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [199]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model
nfl_nn_model = nn.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.8321 - accuracy: 0.5135
Epoch 2/500
5/5 [==============================] - 0s 3ms/step - loss: 0.8033 - accuracy: 0.5135
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 0.7757 - accuracy: 0.5135
Epoch 4/500
5/5 [==============================] - 0s 3ms/step - loss: 0.7577 - accuracy: 0.5135
Epoch 5/500
5/5 [==============================] - 0s 5ms/step - loss: 0.7395 - accuracy: 0.5135
Epoch 6/500
5/5 [==============================] - 0s 6ms/step - loss: 0.7281 - accuracy: 0.5068
Epoch 7/500
5/5 [==============================] - 0s 4ms/step - loss: 0.7176 - accuracy: 0.5000
Epoch 8/500
5/5 [==============================] - 0s 5ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 0.7020 - accuracy: 0.5135
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 0.6960 - accuracy: 0.5270
Epoch 11/500
5/5 [===========

In [200]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 1.6173 - accuracy: 0.4000 - 187ms/epoch - 187ms/step
Loss: 1.617297649383545, Accuracy: 0.4000000059604645


In [201]:
y_pred = nn.predict(X_test_scaled)

y_pred

array([[0.9998144 ],
       [0.1284279 ],
       [0.99970835],
       [0.98090625],
       [0.99467784],
       [0.7584061 ],
       [0.9999882 ],
       [0.9407027 ],
       [0.20399532],
       [0.839608  ],
       [0.97561467],
       [0.848016  ],
       [0.8933537 ],
       [0.1284279 ],
       [0.1284279 ]], dtype=float32)

In [202]:
def get_schedule(year):
    weeks = list(range(1,19))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        week_scores = Boxscores(weeks[w],year)
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game = pd.DataFrame(week_scores.games[date_string][g], index = [0])[['away_name', 'away_abbr','home_name', 'home_abbr','winning_name', 'winning_abbr' ]]
            game['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,game])
        schedule_df = pd.concat([schedule_df, week_games_df]).reset_index().drop(columns = 'index') 
    return schedule_df 

In [203]:
test_df.head()

,win_perc_dif,first_downs_dif,fumbles_dif,interceptions_dif,net_pass_yards_dif,pass_attempts_dif,pass_completions_dif,pass_touchdowns_dif,pass_yards_dif,penalties_dif,points_dif,rush_attempts_dif,rush_touchdowns_dif,rush_yards_dif,time_of_possession_dif,times_sacked_dif,total_yards_dif,turnovers_dif,yards_from_penalties_dif,yards_lost_from_sacks_dif,fourth_down_perc_dif,third_down_perc_dif,result,elo_dif,qb_dif
148,0.300000,-1.600000,0.500000,0.100000,-42.000000,-8.700000,-7.300000,-0.100000,-37.100000,-0.400000,0.300000,4.100000,0.000000,17.900000,-20.600000,0.900000,-24.100000,0.100000,0.500000,4.900000,-7.763975e-02,0.010196,1.0,146.190723,1.927580
149,-0.200000,-4.300000,-0.300000,0.100000,5.300000,-0.100000,-0.500000,-0.600000,8.100000,-0.200000,-7.000000,-6.200000,-0.100000,-50.100000,-113.400000,0.300000,-44.800000,-0.300000,2.500000,2.800000,-9.049774e-03,-0.086816,1.0,-95.585615,-39.040283
150,0.100000,3.300000,0.900000,0.300000,69.900000,7.700000,7.200000,-0.100000,69.300000,1.400000,4.400000,-3.400000,0.300000,0.900000,43.700000,-0.200000,70.800000,0.300000,15.100000,-0.600000,-2.833333e-01,0.104856,1.0,22.731979,131.385591
151,0.154545,1.336364,-0.127273,-0.445455,-30.445455,-6.636364,-5.009091,0.181818,-35.063636,0.709091,8.100000,7.427273,0.845455,60.463636,8.845455,-0.381818,30.018182,-0.581818,4.772727,-4.618182,-1.041667e-01,0.087977,0.0,81.771113,66.528163
152,0.090909,0.818182,-0.090909,0.363636,-10.454545,1.090909,-0.636364,-0.181818,-7.454545,0.818182,-0.818182,2.727273,0.272727,12.545455,116.181818,0.818182,2.272727,0.181818,7.909091,3.000000,-1.110223e-16,-0.052341,0.0,1.748849,34.094641


In [204]:
# Create dataframe to hold prediction results
results_df = test_df_info[['week','away_name','home_name']]
results_df['away_team_win_%'] = y_pred
results_df['home_team_win_%'] = 1 - results_df['away_team_win_%']
results_df['predicted_winner'] = 0

# Loop through results dataframe and add predicted_winner name to column
for index, row in results_df.iterrows():
    if results_df.loc[index,'away_team_win_%'] > results_df.loc[index,'home_team_win_%']:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'away_name']
    else:
        results_df.loc[index,'predicted_winner'] = results_df.loc[index,'home_name']

# Set Index to the 'week' column
results_df=results_df.set_index('week')      

# Use get_schedule function to pull dataframe that shows actual weekly winners. Includes all weeks.
weekly_winner = get_schedule('2021')
weekly_winner = weekly_winner.drop(columns=['away_abbr','home_abbr','winning_abbr'])
weekly_winner = weekly_winner.rename(columns={'away_name':'ww_away_name','home_name':'ww_home_name'})

# Slices weekly_winner dataframe based on current week model is predicting.
current_week_wins = weekly_winner.loc[weekly_winner['week'] == current_week]
current_week_wins = current_week_wins.set_index('week')

# Sort both dataframes by away_name
results_df = results_df.sort_values(by='away_name')
current_week_wins = current_week_wins.sort_values(by='ww_away_name')

# Combine results & weekly_winner df's. Drop dupe columns and rename winning_name to 'actual_winner'
actual_results_df = (pd.concat([results_df,current_week_wins],axis=1)).drop(columns=['ww_away_name','ww_home_name']).rename(columns={'winning_name':'actual_winner'})
actual_results_df


C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\14065\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner
week,,,,,,
12,Atlanta Falcons,Jacksonville Jaguars,0.999988,0.000012,Atlanta Falcons,Atlanta Falcons
12,Buffalo Bills,New Orleans Saints,0.999708,0.000292,Buffalo Bills,Buffalo Bills
12,Carolina Panthers,Miami Dolphins,0.758406,0.241594,Carolina Panthers,Miami Dolphins
12,Chicago Bears,Detroit Lions,0.999814,0.000186,Chicago Bears,Chicago Bears
12,Cleveland Browns,Baltimore Ravens,0.128428,0.871572,Baltimore Ravens,Baltimore Ravens
12,Las Vegas Raiders,Dallas Cowboys,0.128428,0.871572,Dallas Cowboys,Las Vegas Raiders
12,Los Angeles Chargers,Denver Broncos,0.975615,0.024385,Los Angeles Chargers,Denver Broncos
12,Los Angeles Rams,Green Bay Packers,0.893354,0.106646,Los Angeles Rams,Green Bay Packers
12,Minnesota Vikings,San Francisco 49ers,0.848016,0.151984,Minnesota Vikings,San Francisco 49ers


In [205]:
actual_results_df['Accuracy'] = np.where(actual_results_df['predicted_winner'] == actual_results_df['actual_winner'], 1, 0)

actual_results_df = actual_results_df.dropna()

actual_results_df

,away_name,home_name,away_team_win_%,home_team_win_%,predicted_winner,actual_winner,Accuracy
week,,,,,,,
12,Atlanta Falcons,Jacksonville Jaguars,0.999988,0.000012,Atlanta Falcons,Atlanta Falcons,1
12,Buffalo Bills,New Orleans Saints,0.999708,0.000292,Buffalo Bills,Buffalo Bills,1
12,Carolina Panthers,Miami Dolphins,0.758406,0.241594,Carolina Panthers,Miami Dolphins,0
12,Chicago Bears,Detroit Lions,0.999814,0.000186,Chicago Bears,Chicago Bears,1
12,Cleveland Browns,Baltimore Ravens,0.128428,0.871572,Baltimore Ravens,Baltimore Ravens,1
12,Las Vegas Raiders,Dallas Cowboys,0.128428,0.871572,Dallas Cowboys,Las Vegas Raiders,0
12,Los Angeles Chargers,Denver Broncos,0.975615,0.024385,Los Angeles Chargers,Denver Broncos,0
12,Los Angeles Rams,Green Bay Packers,0.893354,0.106646,Los Angeles Rams,Green Bay Packers,0
12,Minnesota Vikings,San Francisco 49ers,0.848016,0.151984,Minnesota Vikings,San Francisco 49ers,0


In [206]:
#Week Accuracy 
accuracy = sum(actual_results_df['Accuracy'])/len(actual_results_df)
round(accuracy,2)

0.4

In [207]:
# Export actual_results_df to csv based on the current_week predicted.
file_path = str(f'Resources/week_{current_week}_base_results.csv')
actual_results_df.to_csv(Path(file_path))